In [ ]:
import skimage
import skimage.io as sio
import matplotlib.pyplot as plt
from skimage import data

## Image Analysis

Images are a common form of data encountered in scientific disciplines from astronomy to medicine to metallurgy and knowing how to work with them is a valuable skillset. Image analysis tools are becoming increasingly powerful allowing researchers to ask new questions and speed us existing workflows. Machine learning methods that we are learning about right now have also been applied to image analysis and can do some things that start to sound like science fiction: facial recognition and robots that can see and interact with their environment. We won't be doing anything on that level here in class, but we will get you on the right track to feeling comfortable with interacting with images in Python and giving you a set of tools to do some introductory analyses of common imaging problems.

In **Lecture 1**, we will learn how to import, save, and interact with image datasets, learn some basics about images, and introduce [**scikit-image**](https://scikit-image.org/) or **skimage**, a collection of tools for image processing workflows.

In **Lecture 2** we will apply the tools from skimage to perform a common image analysis task: image registration and calculation of morphological properties of objects in the image.

## Importing and Saving Images

First off, in order to work with images, we need to be able to read them into our Jupyter notebook. We will interact with images as a variable within our workspace. For the purposes of this demonstration, we will be using images from the `skimage.data module`. We will also import the `skimage.io` module, which has a useful set of functions for importing, displaying, and saving images.

Our first example will use an image of a photographer, and store it as the variable `camera`. 

You may be curious what this object is, so let's check `type(camera)`:

Ah, so `skimage` reads in images as numpy arrays. Digital images in fact are nothing more than an array of pixel intensities, so numpy arrays are a natural medium for image datasets. Other things you may want to check about your image is the datatype of each individal pixel and the size of the image, and maybe check a small subset of the data:

So our image has 512 x 512 pixels, and each pixel is an 8-bit unsigned integer. This means that each pixel can be assigned a value between 0 and $2^8 - 1$. Good to know. But something you may want to do is convert between image types. Why care? Depending on the image type, they will take up different amounts of space.

We can import a set of utility functions from `skimage.util` to do this:

Let's check what difference it makes in file size. We can save to a file using `sio.imsave`:

That's a big difference! There was a 10-fold change in size when we went from uint8 images to float-64 images. Keep that in mind when analyzing datasets, especially if you are dealing with very large images like tilescans or videos.

## Image Compression and Resolution

Now would probably be a good time to talk about image formats too. You noticed that I saved the file with a `.tif` extension. What is that? It may be new to you, it may be not. Perhaps you have used `.png` or `.jpeg` or `.gif` file formats. [Here's](https://www.photoup.net/differences-between-file-formats-raw-dng-tiff-gif-png-jpeg/) a good explanation that talks about the most common types. In summary,

* **TIF** (Tagged Image File Format) is very common in scientific imaging. It is lossless (meaning that you don't lose any information during the compression process). The downside is they are extremely large. They also have compatibility issues when using in HTML.

* **GIF** (Graphics Interchange Format): very small, which is useful for animated images on the internet. Perhaps not the most useful for scientific purposes, because they are extremely lossy.

* **PNG** (Portable Network Graphics): some data compression like GIFS, but no loss of information. Probably a good option for some scientific applications.

* **JPEG** (Joint Photographic Experts Group) renders high quality image, has more compression than PNG, but some loss of information.

But what difference does that make on filesize? Let's check:

It looks like file format can make a between 2- and 10-fold difference in file size. 

The last thing we will look at that can impact file size is image resolution. But before we do that, let's actually look at our image. In the first twenty minutes talking about image analysis, and we haven't even pulled up an image!

Nice. You can adjust the contrast of the image (for display purposes only-- it doesn't impact the data) using the `vmin` and `vmax` arguments

Remember that our original image is 512 x 512 pixels. What if we want to reduce the resolution by, say, 10-fold? We can use a command from the `skimage.transform` module to assist us with that:

Similarly, you can increase the resolution using the `rescale` function. Keep in mind, this won't magically increase the image quality. It will just have a lot of replicate pixels. It can be useful though when you need to map a lower resolution image with a higher resolution image.

How big of an effect do you think image resolution will have on file size?

You can see that by accounting for data type, compression, and resolution, images can $~10^4 - 10^5$ orders of magnitude! Again, something you will definitely need to account for when planning your data storage.

## Working in Color

What if we're working with color images? Well, color images have an extra dimension:

Color images are represented using a **color space**. One of the most common, with which you are probably familiar, is the **RGB space**, with a red, green, and blue channel. We can break the above image into its respective RGB components by selecting the appropriate channel in the numpy array. 

To visualize the RGB space, think of each component as a dimension in a cube:

<img src="https://upload.wikimedia.org/wikipedia/commons/0/05/RGB_Cube_Show_lowgamma_cutout_a.png" width="200"> </img>

But the RGB color space isn't the only one available, and often isn't the most useful. Introducing the HSV color space, which stands for:

* hue
* saturation
* intensity

It has a slightly different geometrical representation:

<img src="https://upload.wikimedia.org/wikipedia/commons/3/33/HSV_color_solid_cylinder_saturation_gray.png" width="200"> </img>

Notice the useful property of HSV that the V element is a black and white rendering of the original image.

## A few basic analysis tools

So far, we have looked at how to work with images examining issues such as input/output, storage, and visualization. But we have yet to actually touch on any aspects of analysis, extracting data from images. Ultimately, we would like to have a numerical set of data that we can manipulate in something like pandas.

Let's look at a very common imaging analysis problem: image segmentation. Image segmentation is the process of labelling groups of pixels of interest within an image that correspond to objects. Sure, you as a human can do this very easily. But its much harder when you want to try to do it programmatically.

Let's pull another example image from the `skimage` library:

In this image, we can see a selection of coins. We know which pixels are coins and which pixels are background. What distinguishes the two? Well, pixel intensities are a good bet. At this point, it would be useful to know what range of pixel values are in the image. Let's make a histogram:

High values correspond to brighter pixels and low values correspond to darker pixels. There seems to be a large peak at ~40 that corresponds to background, and another peak at ~165 that corresponds to the coins. Let's try setting a threshold that binarizes our image into either background or coins. We'll try a few different values to see what happens:

You can see the problem. When we set a low threshold, a lot of the background gets improperly labelled. Due to some uneven illumination, the upper left corner all gets detected. If we set a high threshold, the whole coins don't get labelled. There are lots of holes. And in all cases, there are some objects that definitely aren't coins: a few stray pixels here and there. How do we get a cleaner separation of coins and background?

Well, one good idea would be to to look at pixel intensity gradients instead of absolute values. Visually, you can tell there is a sharp drop at the edges of coins. `skimage` has a tool for edge detection in the `skimage.feature` module called `canny`. For a little background, the `canny` decector performs the following steps:

1. Apply a Gaussian filter to smooth the image in order to remove noise.
2. Find the intensity gradients of the image.
3. Apply non-maximum suppression to get rid of spurious responses (thin the edges).
4. Apply double threshold (low and high thresholds) to determine potential edges.
5. Suppress "weak" edges.

It would be beneficial to talk about each of the substeps of an edge detector, as they can be useful in developing your own imaging workflows. First, Gaussian filters.

When you apply a filter to an image, you are doing some matrix math with a function referred to as a convolution. Think of it as a moving average over the original image. You as the user have two knobs you can turn: the size of the window over which the average is taken, and the contribution of each individual pixel to the final average. This is represented mathematically as a matrix operation:

$A = \begin{bmatrix} 1 & 1 & 1 & 1\\ 1 & 1 & 1 & 1\\ 1 & 1 & 1 & 1\\ 1 & 1 & 1 & 1\end{bmatrix} * B$

In the case when all the values in the matrix are ones, you have applied a mean filter, because each pixel within the "window" contributes equally to the final value. Let's try applying a mean filter of varying sizes to our original image:

Another little nuance is the shape of the structuring element, defined by the variable `selem`:

A Gaussian filter is very similar, but the pixel values within the structuring element are no longer all one. They are weighted, such that they drop off following a Gaussian curve the farther you are away from the the center of the structuring element. Instead of adjusting the size of the structuring element, you adjust the standard deviation of the Gaussian curve.

Even just using a Gaussian filter can improve our segmentation problem. Let's try applying a threshold to the above images:

The next step in the Canny edge detector is an intensity gradient of the input image. There are several different gradient functions available, all using different approximations of the actual gradient, which is computationally expensive. Let's try out a few:

Let's try adding a Gaussian filter, an edge detector, and a threshold all together to make our own mini Canny edge detector:

Let's jump back to our image registration problem. So far, we have been able to detect the edges of the coins using the Canny edge detector. But this isn't actually what we wanted: we wanted to be able to separate the coins out, not their edges. We can solve this problem with a handy hole filler tool called `binary_fill_holes` that's in a separate package, `scipy`.

`binary_fill_holes` works by a clever trick. Think of each white pixel, shape, line, as a wall. In this analogy, `binary_fill_holes` floods the image with water. Anything that doesn't get filled with water is set to `True`. 

Finally, we can use a useful command to clear up any small objects that likely are false positives and not actually coins:

Sweet, right?! We were successfully able to separate the coins from the background. This in itself is a pretty big step, but we still don't have access to the properties of the shapes of the coins themselves. How do we do that? `skimage` has a set of functions in the `measure` module that help us get there.

The first is `label`. `label` does exactly what it says: it labels each unique object in the binary image with a number. It records the label number as a pixel value. 

We can use our labelled image as the input to the function `